#### Gathering data from database

requests; 

 select createdat, deletedat, requestid, tsstart, type, is_passed, leadid from calendar_event
 where createdat > '2025-03-01'
 and createdat < '2025-08-15'
and type <> 'VACATION'

Qualitative info + target column

select r.id as requestid, 
zipregion, r.evaluationtime, r.desiredinstallationend, electricitybill, heatingbill l.mktgparamscore, a.grosscontractsigned,  selfipaimportedat  -- for the two column that are present both in request and lead; request is more filled
from  request r left join lead l on l.id = r.leadid left join airtable_contracts a on a.requestid = r.id
-----NO TIME CONSTRAINT ON THIS -----

In [1]:

import pandas as pd
import numpy as np


In [21]:
# requests new
file_path = r"C:\Users\ElifYilmaz\Downloads\project 2.0.csv"

df = pd.read_csv(file_path)
df.columns = df.columns.str.lower()


In [23]:
# requests contracts
path = r"C:\Users\ElifYilmaz\Downloads\project 2.0_contracts.csv"
df2 = pd.read_csv(path)
df2.columns = df.columns.str.lower()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76097 entries, 0 to 76096
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   createdat  76097 non-null  object
 1   deletedat  32987 non-null  object
 2   requestid  76097 non-null  int64 
 3   tsstart    76097 non-null  object
 4   type       76097 non-null  object
 5   is_passed  76097 non-null  int64 
 6   leadid     76097 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 4.1+ MB


In [27]:
# Check for overlapping IDs
df_combinations = df[['leadid', 'tsstart']].drop_duplicates()
df2_combinations = df2[['leadid', 'tsstart']].drop_duplicates()
overlapping_combinations = pd.merge(df_combinations, df2_combinations, on=['leadid', 'tsstart'], how='inner')

if len(overlapping_combinations) > 0:
    print(f"WARNING: Found {len(overlapping_combinations)} overlapping leadid+tsstart combinations:")
    print(overlapping_combinations)
    
    # Show which specific rows have duplicates
    overlap_mask_df = df[['leadid', 'tsstart']].apply(tuple, axis=1).isin(overlapping_combinations[['leadid', 'tsstart']].apply(tuple, axis=1))
    overlap_mask_df2 = df2[['leadid', 'tsstart']].apply(tuple, axis=1).isin(overlapping_combinations[['leadid', 'tsstart']].apply(tuple, axis=1))
    
    print(f"\nRows in df with overlapping combinations: {overlap_mask_df.sum()}")
    print(f"Rows in df2 with overlapping combinations: {overlap_mask_df2.sum()}")
else:
    print("No overlapping leadid+tsstart combinations found - safe to concatenate!")


No overlapping leadid+tsstart combinations found - safe to concatenate!


In [28]:
combined_df = pd.concat([df, df2], ignore_index=True)


In [29]:
df = combined_df.copy()

In [17]:
# qualitative 

file2 = r"C:\Users\ElifYilmaz\Downloads\info client.csv"

df2 = pd.read_csv(file2)
df2.columns = df2.columns.str.lower()



### duplicates in df2

In [18]:
# Step 1: Count requestids that have duplicates
duplicate_requestids = df2[df2['requestid'].duplicated(keep=False)]['requestid'].unique()
print(f"Number of requestids with duplicates: {len(duplicate_requestids)}")
print(f"Total duplicate rows: {df2['requestid'].duplicated().sum()}")

# Step 2: Drop absolute duplicates (excluding leadid column)
comparison_cols = [col for col in df2.columns if col != 'leadid']
df2_before = len(df2)
df2 = df2.drop_duplicates(subset=comparison_cols, keep='first')
df2_after = len(df2)

print(f"\nDropped {df2_before - df2_after} absolute duplicate rows")

# Step 3: Show differences for remaining duplicate requestids
remaining_duplicates = df2[df2['requestid'].duplicated(keep=False)]

if len(remaining_duplicates) > 0:
    print(f"\nRemaining duplicate requestids: {df2['requestid'].duplicated().sum()}")
    print(f"Number of requestids still with duplicates: {remaining_duplicates['requestid'].nunique()}")
    
    # Analyze differences for each duplicate requestid
    still_duplicate_ids = remaining_duplicates['requestid'].unique()
    
    print(f"\n🔍 DIFFERENCES ANALYSIS:")
    print("=" * 50)
    
    for requestid in still_duplicate_ids[:5]:  # Show first 5 to avoid overwhelming output
        print(f"\nRequestID: {requestid}")
        print("-" * 30)
        
        # Get all rows for this requestid
        group = df2[df2['requestid'] == requestid]
        
        # Find columns that differ across rows
        differing_columns = []
        for col in df2.columns:
            if col != 'requestid':  # Skip the grouping column
                unique_values = group[col].nunique()
                if unique_values > 1:
                    differing_columns.append(col)
        
        if differing_columns:
            print(f"Columns that differ: {differing_columns}")
            # Show only the differing columns + requestid
            cols_to_show = ['requestid'] + differing_columns
            print(group[cols_to_show].to_string(index=False))
        else:
            print("No differences found (should not happen)")
        
        print()
    
    if len(still_duplicate_ids) > 5:
        print(f"... and {len(still_duplicate_ids) - 5} more duplicate requestids")

else:
    print(f"\n✅ No remaining duplicates! All duplicates were absolute duplicates and have been removed.")

Number of requestids with duplicates: 35
Total duplicate rows: 35

Dropped 8 absolute duplicate rows

Remaining duplicate requestids: 27
Number of requestids still with duplicates: 27

🔍 DIFFERENCES ANALYSIS:

RequestID: 21540
------------------------------
Columns that differ: ['netcontractsigned']
 requestid  netcontractsigned
     21540                1.0
     21540                0.0


RequestID: 84726
------------------------------
Columns that differ: ['netcontractsigned']
 requestid  netcontractsigned
     84726                1.0
     84726                0.0


RequestID: 86369
------------------------------
Columns that differ: ['netcontractsigned']
 requestid  netcontractsigned
     86369                1.0
     86369                0.0


RequestID: 88220
------------------------------
Columns that differ: ['netcontractsigned']
 requestid  netcontractsigned
     88220                1.0
     88220                0.0


RequestID: 105757
------------------------------
Columns t

In [ ]:
# they are in net contrcact signed.. double entries in airtable contracts. will handle in notebook 02

In [4]:
# merge for analysis

final_df = df.merge(df2, on='requestid', how='left')

In [20]:
# duplicates in final_df? --> we expect it because now df is a list of transactions.
# Show which requestid values appear more than once
duplicate_requestids = final_df[final_df['requestid'].duplicated(keep=False)]['requestid'].unique()
print(f"Duplicate requestid values: {duplicate_requestids}")

Duplicate requestid values: [106800 106797 106801 ... 122658 122670 122706]


Explatory checks

In [5]:
# basic check, correlation can only be checked after encoding since most values are categorical and the rest are entries of happenings

# data types
print("Data types:")
print(final_df.dtypes)
print(f"\nAny missing values: {final_df.isnull().sum().sum()}")

# shape
print(f"Shape of dataset: {final_df.shape}")


Data types:
createdat                  object
deletedat                  object
requestid                   int64
tsstart                    object
type                       object
is_passed                   int64
leadid                      int64
zipregion                  object
evaluationtime             object
desiredinstallationend     object
electricitybill           float64
heatingbill               float64
mktgparamscore             object
netcontractsigned         float64
selfipaimportedat          object
dtype: object

Any missing values: 249807
Shape of dataset: (62491, 15)


In [6]:
# Check missing values by column
print("Missing values per column:")
missing_counts = final_df.isnull().sum()
print(missing_counts[missing_counts > 0])

# Check the object columns that need encoding
object_cols = final_df.select_dtypes(include=['object']).columns
print(f"\nObject columns needing encoding: {list(object_cols)}")

Missing values per column:
deletedat                 34889
zipregion                  3781
evaluationtime              643
desiredinstallationend      636
electricitybill           23251
heatingbill               47678
mktgparamscore            19820
netcontractsigned         60328
selfipaimportedat         58781
dtype: int64

Object columns needing encoding: ['createdat', 'deletedat', 'tsstart', 'type', 'zipregion', 'evaluationtime', 'desiredinstallationend', 'mktgparamscore', 'selfipaimportedat']


In [7]:
# categorical values
# Check unique values in categorical columns
for col in ['zipregion', 'evaluationtime', 'desiredinstallationend', 'mktgparamscore']:
    unique_count = final_df[col].nunique()
    print(f"{col}: {unique_count} unique values")
    if unique_count < 20:  # Show values if not too many
        print(f"  Values: {sorted(final_df[col].dropna().unique())}")
    print()

zipregion: 20 unique values

evaluationtime: 10 unique values
  Values: ['3-6 mesi', '<3 mesi', '>6 mesi', 'curious', 'evaluation', 'less_than_three_months', 'more_than_six_months', 'three_to_six_months', 'understand_need', 'understand_purchase']

desiredinstallationend: 8 unique values
  Values: ['1-2mesi', '3-4mesi', '5+mesi', 'Non lo so', 'dont_know', 'more_than_5_months', 'one_to_two_months', 'three_to_four_months']

mktgparamscore: 13 unique values
  Values: ['Affiliation', 'Google', 'Mediago', 'Meta', 'Organic', 'Other', 'Outbrain', 'Referral', 'Taboola', 'TikTok', 'Youtube', 'd2d', 'form_classico']



In [8]:
# Check if missingness patterns relate to conversion
print("Conversion rates by missingness:")
for col in ['zipregion', 'evaluationtime', 'desiredinstallationend', 'electricitybill', 'mktgparamscore']:
    missing_conversion = final_df[final_df[col].isnull()]['netcontractsigned'].mean()
    present_conversion = final_df[final_df[col].notnull()]['netcontractsigned'].mean()
    
    print(f"{col}:")
    print(f"  Missing: {missing_conversion:.3f} ({final_df[col].isnull().sum()} cases)")
    print(f"  Present: {present_conversion:.3f} ({final_df[col].notnull().sum()} cases)")
    print()

Conversion rates by missingness:
zipregion:
  Missing: 0.752 (3781 cases)
  Present: 0.763 (58710 cases)

evaluationtime:
  Missing: 0.153 (643 cases)
  Present: 0.779 (61848 cases)

desiredinstallationend:
  Missing: 0.148 (636 cases)
  Present: 0.780 (61855 cases)

electricitybill:
  Missing: 0.777 (23251 cases)
  Present: 0.758 (39240 cases)

mktgparamscore:
  Missing: 0.786 (19820 cases)
  Present: 0.752 (42671 cases)

